In [1]:
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 5.1 MB/s 


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image
from torchvision import transforms

In [8]:
folder_path = "/content/drive/MyDrive/DRAC2022/OCTA"

In [ ]:
!wget https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-resnest/resnest50-528c19ca.pth -P /content/drive/MyDrive/DRAC2022/OCTA/OCTA-Net-OCTA-Vessel-Segmentation-Network-master/code

In [ ]:
%cd {folder_path}/OCTA-Net-OCTA-Vessel-Segmentation-Network-master/code/OCTA-Net

In [ ]:
!python front_main.py --mode train --dataset cria --data_dir ../../../../data/ROSE-2 --val_epoch_freq 5 --num_workers 2

Görüntülenen çıkış son 5000 satıra kısaltıldı.
4 / 45, train loss: 0.1503
5 / 45, train loss: 0.1414
6 / 45, train loss: 0.1003
7 / 45, train loss: 0.1053
8 / 45, train loss: 0.0909
9 / 45, train loss: 0.0999
10 / 45, train loss: 0.0716
11 / 45, train loss: 0.1108
12 / 45, train loss: 0.0560
13 / 45, train loss: 0.1279
14 / 45, train loss: 0.0872
15 / 45, train loss: 0.1397
16 / 45, train loss: 0.1180
17 / 45, train loss: 0.1112
18 / 45, train loss: 0.1591
19 / 45, train loss: 0.1146
20 / 45, train loss: 0.0834
21 / 45, train loss: 0.1025
22 / 45, train loss: 0.0805
23 / 45, train loss: 0.1204
24 / 45, train loss: 0.1205
25 / 45, train loss: 0.1534
26 / 45, train loss: 0.1134
27 / 45, train loss: 0.1296
28 / 45, train loss: 0.1620
29 / 45, train loss: 0.1243
30 / 45, train loss: 0.0912
31 / 45, train loss: 0.1464
32 / 45, train loss: 0.0667
33 / 45, train loss: 0.1180
34 / 45, train loss: 0.1425
35 / 45, train loss: 0.1225
36 / 45, train loss: 0.0662
37 / 45, train loss: 0.1023
38 / 45

In [17]:
!python front_main.py --mode test --dataset cria --data_dir ../../../../data/ROSE-2 --num_workers 2 --first_suffix best_fusion.pth

Start testing...
/content/drive/MyDrive/rose/OCTA-Net-OCTA-Vessel-Segmentation-Network-master/code/OCTA-Net/test.py:75: RuntimeWarning: Mean of empty slice.
  print("Loss - mean: " + str(loss_arr.mean()) + "\tstd: " + str(loss_arr.std()))
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:263: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
Loss - mean: nan	std: nan
AUC - mean: 0.8606882219827194	std: 0.04580924419619761
ACC - mean: 0.9035011638294567	std: 0.034684320848026716
SEN 

In [ ]:
def mkdir(path):
    path = path.strip()
    path = path.rstrip("\\")

    isExists = os.path.exists(path)

    if not isExists:
        os.makedirs(path)
        return True
    else:
        return False

In [25]:
pred_path = 'results/cria/first_stage/fusion/Thresh/'
gt_path = '../../../../data/ROSE-2/test/gt/'
ori_path = '../../../../data/ROSE-2/test/original/'
save_dir = '../../../OctaRoseResults/'
imgs = os.listdir(pred_path)
mkdir(folder_path + '/OctaRoseResults')
for img_path in imgs:
    if not img_path.endswith('.png'):
        continue
    img = cv2.imread(pred_path + img_path)
    pink = (255,0,255)
    white_pixels = np.where(
        (img[:, :, 0] == 255) & 
        (img[:, :, 1] == 255) & 
        (img[:, :, 2] == 255)
    )
    img[white_pixels] = pink

    gt = cv2.imread(gt_path + img_path)
    ori = cv2.imread(ori_path + img_path)

    res = np.zeros((512, 512*3, 3))
    res[:,0:512,:] = gt
    res[:,512:2*512, :] = ori
    res[:,2*512:, :] = img
    cv2.imwrite(save_dir + img_path, res.astype(int))

In [ ]:
import torch

device = torch.device('cuda')
net = torch.load("models/cria/first_stage/front_model-best_fusion.pth").to(device)
net.eval()

In [26]:
ori_path = '../../../../A. Segmentation/1. Original Images/a. Training Set/'
save_dir = '../../../OctaDracResults/'
imgs = os.listdir(ori_path)
mkdir(folder_path + '/OctaDracResults')
for img_path in imgs:
    if not img_path.endswith('.png'):
        continue

    simple_transform = transforms.ToTensor()
    
    ori = Image.open(ori_path + img_path).resize((512,512))
    ori = ori.convert("RGB")
    ori = simple_transform(ori).unsqueeze(0)

    thick_pred, thin_pred, pred = net(ori.to(device))
    pred_arr = pred.squeeze().detach().cpu().numpy()
    pred_img = np.array(pred_arr * 255, np.uint8)
    _, img = cv2.threshold(pred_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    pink = (255,0,255)
    img = np.expand_dims(img, 2)
    img = np.repeat(img, 3, 2)
    white_pixels = np.where(
        (img[:, :, 0] == 255) & 
        (img[:, :, 1] == 255) & 
        (img[:, :, 2] == 255)
    )
    img[white_pixels] = pink


    res = np.zeros((512, 512*2, 3))
    res[:,:512, :] = cv2.resize(cv2.imread(ori_path + img_path), (512,512))
    res[:, 512:, :] = img
    cv2.imwrite(save_dir + img_path, res.astype(int))